<a href="https://colab.research.google.com/github/nalgo-intern/team-b/blob/master/%E3%83%9D%E3%82%B1%E3%83%A2%E3%83%B3%E3%82%BF%E3%82%A4%E3%83%97%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#driveからデータを取得

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#先にGoogle Driveで https://drive.google.com/drive/folders/1DxqM2rP85xaSXRiRfdnLjIJgA-4_U112?usp=sharing をマイドライブにコピーしておくこと 
!cp -R /content/drive/My\ Drive/data /content/
!mkdir learned_data

#水増し

In [3]:
#画像の水増し

import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

# 画像を拡張する関数
def draw_images(generator, x, dir_name, index):
    save_name = 'extened-' + str(index)
    g = generator.flow(x, batch_size=1, save_to_dir=output_dir,
                       save_prefix=save_name, save_format='jpg')

    # 1つの入力画像から何枚拡張するかを指定（今回は50枚）
    for i in range(50):
        bach = g.next()

types =["normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground", "flying", "psychic", "bug", "rock", "ghost", "dragon"]
base_dir = './data/'
for i in types:
    print(i)
    # 出力先ディレクトリの設定
    output_dir = base_dir + i
    
    if not(os.path.exists(output_dir)):
        os.mkdir(output_dir)
    
    # 拡張する画像の読み込み
    images = glob.glob(os.path.join(base_dir + i, "*.jpg"))
    
    # ImageDataGeneratorを定義
    datagen = ImageDataGenerator(rotation_range=30,
                                width_shift_range=20,
                                height_shift_range=0.,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True)
    
    # 読み込んだ画像を順に拡張
    for i in range(len(images)):
        img = load_img(images[i])
        img = img.resize((250, 250))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        draw_images(datagen, x, output_dir, i)

Using TensorFlow backend.


normal
fire
water
grass
electric
ice
fighting
poison
ground
flying
psychic
bug
rock
ghost
dragon


#データセットを作成

In [4]:
!pip install -U numpy

     |████████████████████████████████| 20.4MB 3.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [0]:
from PIL import Image
import os, glob
import numpy as np
import random, math

root_dir = "./data"
types =["normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground", "flying", "psychic", "bug", "rock", "ghost", "dragon"]

X = []
Y = []

def make_sample(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sample(cat, fname)
    return np.array(X), np.array(Y)


def add_sample(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((250, 250))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

allfiles = []

for idx, cat in enumerate(types):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test = allfiles[th:]
X_train, y_train = make_sample(train)
X_test, y_test = make_sample(test)
xy = (X_train, X_test, y_train, y_test)

np.save("./dataset.npy", xy)

#学習

In [2]:
!pip install numpy==1.16.2

     |████████████████████████████████| 17.3MB 3.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.0
    Uninstalling numpy-1.17.0:
      Successfully uninstalled numpy-1.17.0


In [0]:
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

num_classes = 15
im_rows = 250
im_cols = 250
in_shape = (im_rows, im_cols, 3)

# データを読み込む --- (*1)
X_train, X_test, y_train, y_test = np.load('./dataset.npy')

# データを正規化 --- (*2)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# ラベルデータをOne-Hot形式に変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# モデルを定義 --- (*3)
# 51行目のDenseの値を変えてみたり35-40行目を増やしたり減らしたり、Conv2Dの最初の引数を変えたり、Dropoutの引数を変えたり
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=in_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#model.add(Conv2D(128, (3, 3), padding='same'))
#model.add(Activation('relu'))
#model.add(Conv2D(128, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(512))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# モデルをコンパイル --- (*4)
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# 学習を実行 --- (*5)
#batch_sizeやepochsの数を変えたり
hist = model.fit(X_train, y_train,
    batch_size=128, epochs=20,
    verbose=1,
    validation_data=(X_test, y_test))

# モデルを評価 --- (*6)
score = model.evaluate(X_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])

# 学習の様子をグラフへ描画 --- (*7)
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#モデル・重みの保存
json_string = model.model.to_json()
open('./learned_data/data.json', 'w').write(json_string)

hdf5_file = "./learned_data/data.hdf5"
model.model.save_weights(hdf5_file)

Using TensorFlow backend.
W0823 06:47:14.345646 139793009239936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 06:47:14.415095 139793009239936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 06:47:14.435754 139793009239936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 06:47:14.491797 139793009239936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0823 06:47:14.499227 1397930

Train on 17620 samples, validate on 4405 samples
Epoch 1/20
17620/17620 [==============================] - 181s 10ms/step - loss: 2.2388 - acc: 0.2491 - val_loss: 1.7116 - val_acc: 0.3775
Epoch 2/20
17620/17620 [==============================] - 164s 9ms/step - loss: 1.6276 - acc: 0.3992 - val_loss: 1.3655 - val_acc: 0.5106
Epoch 3/20
17620/17620 [==============================] - 163s 9ms/step - loss: 1.3632 - acc: 0.4799 - val_loss: 1.0399 - val_acc: 0.6023
Epoch 4/20
17620/17620 [==============================] - 163s 9ms/step - loss: 1.1274 - acc: 0.5563 - val_loss: 0.8489 - val_acc: 0.6638
Epoch 5/20
17620/17620 [==============================] - 164s 9ms/step - loss: 0.9584 - acc: 0.6222 - val_loss: 0.7763 - val_acc: 0.6899
Epoch 6/20
17620/17620 [==============================] - 164s 9ms/step - loss: 0.8355 - acc: 0.6668 - val_loss: 0.6155 - val_acc: 0.7532
Epoch 7/20
17620/17620 [==============================] - 164s 9ms/step - loss: 0.7663 - acc: 0.6900 - val_loss: 0.5784 - 

#サーバーを起動

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir uploads

--2019-08-23 04:45:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.3.255, 52.2.175.150, 34.202.125.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.3.255|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  75.8MB/s    in 0.2s    

2019-08-23 04:45:43 (75.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
import os
import json
# request フォームから送信した情報を扱うためのモジュール
# redirect  ページの移動
# url_for アドレス遷移
from flask import Flask, request, redirect, url_for
# ファイル名をチェックする関数
from werkzeug.utils import secure_filename
# 画像のダウンロード
from flask import send_from_directory

from keras import models
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json
from keras import backend as K

# 画像のアップロード先のディレクトリ
UPLOAD_FOLDER = './uploads'
# アップロードされる拡張子の制限
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'gif'])

app = Flask(__name__)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

#推測部分
def function(img_path):    
    K.clear_session()
    types = ["ノーマル", "ほのお", "みず", "くさ", "でんき", "こおり", "かくとう", "どく", "じめん", "ひこう", "エスパー", "むし", "いわ", "ゴースト", "ドラゴン"]

    model = model_from_json(open('./learned_data/data.json').read())
    model.load_weights('./learned_data/data.hdf5')

    img_path = img_path 
    img = image.load_img(img_path, target_size=(250,250,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    features = model.predict(x)

    print (features)
    for i in range(0,14):
        if features[0,i] == 1:
            break
    
    return types[i]

def allwed_file(filename):
    # .があるかどうかのチェックと、拡張子の確認
    # OKなら１、だめなら0
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

    # ファイルを受け取る方法の指定
@app.route('/', methods=['GET', 'POST'])
def uploads_file():
    # リクエストがポストかどうかの判別
    if request.method == 'POST':
        # ファイルがなかった場合の処理
        if 'file' not in request.files:
            # flash('ファイルがありません')
            return redirect(request.url)
        # データの取り出し
        file = request.files['file']
        # ファイル名がなかった時の処理
        if file.filename == '':
            # flash('ファイルがありません')
            return redirect(request.url)
        # ファイルのチェック
        if file and allwed_file(file.filename):
            # 危険な文字を削除（サニタイズ処理）
            global filename
            filename = secure_filename(file.filename)
            # ファイルの保存
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            # アップロード後のページに転送
            print(file.filename)
            print(filename)
            
            # 関数呼び出し ポケモンの属性を文字で返す
            target_path = "./uploads/" + filename
            global result_data
            result_data = function(target_path)

            # ポケモンの属性を格納する変数
            #result_data = target_path

            return redirect(url_for('result'))

    return '''
    <!doctype html>
    <html>
        <head>
            <meta charset="UTF-8">
            <title>
                ポケモンタイプ診断
            </title>
        </head>
        <body>
            <h1>
                ファイルをアップロードしてポケモンのタイプを判定しよう
            </h1>
            <form method = post enctype = multipart/form-data>
            <p><input type=file id="q" name = file>
            <input type = submit id="qButton" value = 調べる>
            <div id="result"></div>
            </form>

            <script>
                const qs = (q) => document.querySelector(q)
                window.onload = () => {
                const q = qs('#q')
                const qButton = qs('#qButton')
                const result = qs('#result')
                // 判定ボタンを押した時 --- (*1)
                qButton.onclick = () => {
                    result.innerHTML = "..."
                    // APIサーバに送信するURLを構築 --- (*2)
                    const api = "/api?q=" + 
                    encodeURIComponent(q.value)
                    // APIにアクセス --- (*3)
                    fetch(api).then((res) => {
                    return res.json() // JSONで返す
                    }).then((data) => {
                    // 結果を画面に表示 --- (*4)
                    result.innerHTML =
                        data["label"] + 
                        "<span style='font-size:0.5em'>(" + 
                        data["per"] + ")</span>"
                    })
                }
                }
            </script>
        </body>
'''

@app.route('/uploads/<filename>')
# ファイルを表示する
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# resultした場合 
@app.route('/result', methods=['GET'])
def result():
    # img_path = r'C:\Users\internship\team_b\team-b-master\FLASK_UPLOADER\src\pokemon.jpg'
    #     <img src=> 
    # <img src=./src/pokemon.jpg> 

    result_format = """
    <!DOCTYPE html>
    <html>
    <head><meta charset="UTF-8"><title>ポケモンタイプ</title></head>
    <body>

    <p>
        <h1>結果画面</h1>
        <h2>ポケモンのタイプは{%s}です</h2>

    </p>
    </body>
    </html>
    """%result_data
    return result_format 

if __name__ == '__main__':
    app.run(port=6006)

https://1617ac19.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0823 04:47:07.873709 140317138327424 _internal.py:122]  * Running on http://127.0.0.1:6006/ (Press CTRL+C to quit)
I0823 04:47:10.999347 140314312046336 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:47:10] "GET / HTTP/1.1" 200 -
I0823 04:47:11.260983 140314312046336 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:47:11] "GET /favicon.ico HTTP/1.1" 404 -
I0823 04:47:14.013211 140314312046336 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:47:14] "GET /api?q=C%3A%5Cfakepath%5Ctest.jpg HTTP/1.1" 404 -


test.jpg
test.jpg


I0823 04:47:15.082012 140314312046336 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:47:15] "POST / HTTP/1.1" 302 -


[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


...

I0823 04:49:00.587077 140314216023808 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:49:00] "GET /result HTTP/1.1" 200 -
I0823 04:49:02.274737 140314216023808 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:49:02] "GET / HTTP/1.1" 200 -
I0823 04:49:33.649118 140314216023808 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:49:33] "GET /api?q=C%3A%5Cfakepath%5Ctest1.jpg HTTP/1.1" 404 -


test1.jpg
test1.jpg


I0823 04:49:34.371789 140314216023808 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:49:34] "POST / HTTP/1.1" 302 -


[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


I0823 04:49:34.587304 140314216023808 _internal.py:122] 127.0.0.1 - - [23/Aug/2019 04:49:34] "GET /result HTTP/1.1" 200 -


#バックアップ

In [0]:
!cp -R ./learned_data ./drive/My\ Drive
!cp dataset.npy ./drive/My\ Drive